In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') > DESIRED_ACCURACY):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2020-09-10 19:13:49--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.107.128, 74.125.142.128, 74.125.195.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.107.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2020-09-10 19:13:49 (108 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [2]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr = 0.001), metrics=['acc'])

In [3]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.0)

train_generator = train_datagen.flow_from_directory(
    '/tmp/h-or-s/', 
    target_size=(150,150), 
    batch_size=32, 
    class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [6]:
# This code block should call model.fit_generator and train for
# a number of epochs. 
history = model.fit(
    train_generator,
    steps_per_epoch=1,  
    epochs=15,
    verbose=1, 
    callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
1/1 [==============================] - 0s 2ms/step - loss: 0.6767 - acc: 0.5938
Epoch 2/15
1/1 [==============================] - 0s 2ms/step - loss: 0.6990 - acc: 0.5625
Epoch 3/15
1/1 [==============================] - 0s 6ms/step - loss: 0.6494 - acc: 0.6250
Epoch 4/15
1/1 [==============================] - 0s 2ms/step - loss: 0.6497 - acc: 0.5625
Epoch 5/15
1/1 [==============================] - 0s 6ms/step - loss: 0.8476 - acc: 0.5000
Epoch 6/15
1/1 [==============================] - 0s 6ms/step - loss: 0.6385 - acc: 0.9062
Epoch 7/15
1/1 [==============================] - 0s 5ms/step - loss: 0.5802 - acc: 0.6562
Epoch 8/15
1/1 [==============================] - ETA: 0s - loss: 0.5138 - acc: 1.0000
Reached 99.9% accuracy so cancelling training!
1/1 [==============================] - 0s 2ms/step - loss: 0.5138 - acc: 1.0000
